In [2]:
xeus_source = 'F32_PROD_XEUS'


In [ ]:
query = '''
    SELECT DISTINCT
        *
    FROM
    
    WHERE
'''

In [10]:
with PyUber.connect(datasource=xeus_source) as conn:
    df = pd.read_sql(query, conn)
df

C:\Users\dagarcia\AppData\Local\Temp\ipykernel_36676\3925277670.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,OUT_WW,WW_START_TIME,WW_END_TIME,OPERATION,OPER_SHORT_DESC,AREA,MODULE,PRODUCT,LOT_DOTPROCESS,LOT_TYPE,...,TOTAL_DIE_MOVEOUT_IN_REWORK,TOTAL_DIE_SKIPPED,TOTAL_DIE_OUTS,TOTAL_DIE_REWORK_OUTS,GOOD_DIE_MOVEOUT,GOOD_DIE_MOVEOUT_LAST_PASS,GOOD_DIE_MOVEOUT_IN_REWORK,GOOD_DIE_SKIPPED,GOOD_DIE_OUTS,GOOD_DIE_REWORK_OUTS
0,202529.0,2025-07-12 18:00:00,2025-07-19 17:59:59,228293,A-8DPRHM2c,ANALYTICAL,AN-NCD,8PXMCV A G,1278.3,TD,...,None,None,None,None,None,None,None,None,None,None
1,202529.0,2025-07-12 18:00:00,2025-07-19 17:59:59,204139,L-8NDRbaFC,LITHO,LI-SEMli,8PXMCV A G,1278.3,TD,...,None,None,None,None,None,None,None,None,None,None
2,202528.0,2025-07-05 18:00:00,2025-07-12 17:59:59,202263,D*8ZROXPAD,DIFFUSION,DI-VFOae,8PXMCV A G,1278.3,TD,...,None,None,None,None,None,None,None,None,None,None
3,202528.0,2025-07-05 18:00:00,2025-07-12 17:59:59,204056,L 8ZRObaSE,LITHO,LI-SNEli,8PXMCV A G,1278.3,TD,...,None,None,None,None,None,None,None,None,None,None
4,202528.0,2025-07-05 18:00:00,2025-07-12 17:59:59,256844,E 8ZRORWK1,ETCH,DE-GTArk,8PXMCV A G,1278.3,TD,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1256,202531.0,2025-07-26 18:00:00,2025-08-02 17:59:59,263383,E 8OXEPIRW,ETCH,DE-OXTrk,8PXMCV A G,1278.3,TD,...,None,None,None,None,None,None,None,None,None,None
1257,202531.0,2025-07-26 18:00:00,2025-08-02 17:59:59,249454,W*8FNHMaCL,WET ETCH,WE-NSTwp,8PXMCV A G,1278.3,PROD,...,None,None,None,None,None,None,None,None,None,None
1258,202531.0,2025-07-26 18:00:00,2025-08-02 17:59:59,251549,A-8STIR2N,ANALYTICAL,AN-NCD,8PXMCV A G,1278.3,TD,...,None,None,None,None,None,None,None,None,None,None
1259,202531.0,2025-07-26 18:00:00,2025-08-02 17:59:59,198979,W*8TEDPRC4,WET ETCH,WE-NSTwp,8PXMCV A G,1278.3,TD,...,None,None,None,None,None,None,None,None,None,None


In [14]:
df = df[df['LOT_TITLE'].str.startswith('NPI')]